# W207 Project Final - Spring 2017

### **Group:**

Nichole Haas

Jayashree Raman

David Skarbrevik

### **Dataset:**

"Random Acts of Pizza"

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
import json
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit
# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *
import sys
from subprocess import check_output
print(sys.version)

#make the depreciation warnings go away
import warnings
#I'm tired of the warnings on functions the professor asks us to use :) 
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning)

3.5.2 |Anaconda 4.1.1 (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/nicholeh/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
### get the training data
train_json_array = np.array(pd.read_json('train.json', orient='columns'))
#make it sklearn-able
data_set = train_json_array[:,6]
data_set_title = train_json_array[:,8]
data_labels = train_json_array[:,22]

In [3]:
a = pd.read_json('train.json')
for x, y in enumerate(a.columns):
    print(x,y)
#data_set = a['request_text']
#data_set_title = a['request_title']
#data_labels = a['requester_received_pizza']

0 giver_username_if_known
1 number_of_downvotes_of_request_at_retrieval
2 number_of_upvotes_of_request_at_retrieval
3 post_was_edited
4 request_id
5 request_number_of_comments_at_retrieval
6 request_text
7 request_text_edit_aware
8 request_title
9 requester_account_age_in_days_at_request
10 requester_account_age_in_days_at_retrieval
11 requester_days_since_first_post_on_raop_at_request
12 requester_days_since_first_post_on_raop_at_retrieval
13 requester_number_of_comments_at_request
14 requester_number_of_comments_at_retrieval
15 requester_number_of_comments_in_raop_at_request
16 requester_number_of_comments_in_raop_at_retrieval
17 requester_number_of_posts_at_request
18 requester_number_of_posts_at_retrieval
19 requester_number_of_posts_on_raop_at_request
20 requester_number_of_posts_on_raop_at_retrieval
21 requester_number_of_subreddits_at_request
22 requester_received_pizza
23 requester_subreddits_at_request
24 requester_upvotes_minus_downvotes_at_request
25 requester_upvotes_minus_

In [4]:
data_set = a[['request_text_edit_aware','request_title']]
#data_set_title = a['request_title']
data_labels = a['requester_received_pizza']
clf = Pipeline([
    ('counter', CountVectorizer()),
    ('mnb', MultinomialNB())])

sss = StratifiedShuffleSplit(test_size=0.25, random_state=1)
for train_index, test_index in sss.split(data_set, data_labels):
    X_train, X_test = data_set.iloc[train_index], data_set.iloc[test_index]
    y_train, y_test = data_labels.iloc[train_index], data_labels.iloc[test_index]

In [5]:
X_train.shape

(3030, 2)

In [6]:
X_test.shape

(1010, 2)

In [7]:
y_test.value_counts()

False    762
True     248
Name: requester_received_pizza, dtype: int64

In [12]:
X_test.iloc[0]

request_text_edit_aware    Just finished up the last of my classes and ru...
request_title              [REQUEST] Toronto Ontario. Broke student could...
Name: 1177, dtype: object

In [38]:
y_train.value_counts()

False    2284
True      746
Name: requester_received_pizza, dtype: int64

In [13]:
#from http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
#try the Pipeline implementation from sk learn

import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]


class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features


pipeline = Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
                ('cv', CountVectorizer(analyzer='char_wb', 
                                       #vocabulary=vocabulary,
                                       max_df=0.5, ngram_range=(1,3)))
            ])),

            # Pipeline for standard bag-of-words model for body
            ('get-request', Pipeline([
                ('selector', ItemSelector(key='request_text_edit_aware')),
                ('cv', CountVectorizer(analyzer='char_wb', 
                                       #vocabulary=vocabulary,
                                       max_df=0.5, ngram_range=(1,3)))
            ])),
            
           #('get-number', Pipeline([
           #     ('selector', ItemSelector(key='number'))
           # ]))

        ]
    )),

    # Use a SVC classifier on the combined features
    ('nb',LogisticRegression(penalty='l2',C=100)),
])



In [14]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
target_names = ['Got pizza', 'No pizza']
print(classification_report(y_test, y_pred, target_names=target_names))

             precision    recall  f1-score   support

  Got pizza       0.77      0.79      0.78       762
   No pizza       0.31      0.29      0.30       248

avg / total       0.66      0.67      0.66      1010



In [33]:
X_train.shape

(4040, 2)

In [15]:
#from http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
def preprocess(x):
    """Use a series of regex expressions to remove unwanted characters"""
    #remove non-alpha-numeric characters, replace with whitespace
    x1 = re.sub(r'[^a-zA-Z_0-9_\s]'," ", x).lower()
    #replae all numbers with a single token and a space afterwards
    x1a = re.sub(r'[0-9]+', 'number ', x1)
    #x1b = re.sub(r'[_]+', ' ', x1a)
    #even though there are words that are just '_____', f1 actuall decreases when they're removed
    #remove newlines
    x2 = re.sub(r'[\n]', " ", x1a)
    #scrub out extra spaces
    x3 = re.sub(r'\s+', ' ', x2)  #other steps might have added extra space; remove
    return x3.strip()


import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]


class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features


pipeline = Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
                ('cv', CountVectorizer(preprocessor = preprocess,
                                       analyzer='char_wb', 
                                       max_df=0.5, ngram_range=(1,3)))
            ])),

            # Pipeline for standard bag-of-words model for body
            ('get-request', Pipeline([
                ('selector', ItemSelector(key='request_text_edit_aware')),
                ('cv', CountVectorizer(preprocessor = preprocess, 
                                       analyzer='char_wb', 
                                       max_df=0.5, ngram_range=(1,3)))
            ])),
            
           #('get-number', Pipeline([
           #     ('selector', ItemSelector(key='number'))
           # ]))

        ]
    )),

    # Use a SVC classifier on the combined features
    ('nb',LogisticRegression(penalty='l2',C=100)),
])



In [16]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      False       0.76      0.76      0.76       762
       True       0.27      0.27      0.27       248

avg / total       0.64      0.64      0.64      1010



In [50]:
# I had hypothesized earlier that emojis would matter, so we ought not to 
# strip them.  the preprocessor used above strips emojis, and you can see
# the predictor didn't perfom as well.  

In [17]:
#from http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
def preprocess(x):
    """Use a series of regex expressions to remove unwanted characters"""
    #remove non-alpha-numeric characters, replace with whitespace
    x1 = x.lower()
    #replae all numbers with a single token and a space afterwards
    x1a = re.sub(r'[0-9]+', 'number ', x1)
    #x1b = re.sub(r'[_]+', ' ', x1a)
    #even though there are words that are just '_____', f1 actuall decreases when they're removed
    #remove newlines
    x2 = re.sub(r'[\n]', " ", x1a)
    #scrub out extra spaces
    x3 = re.sub(r'\s+', ' ', x2)  #other steps might have added extra space; remove
    return x3.strip()


import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
#from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC


class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]


class TextStats(BaseEstimator, TransformerMixin):
    """Extract features from each document for DictVectorizer"""

    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        return [{'length': len(text),
                 'num_sentences': text.count('.')}
                for text in posts]


class SubjectBodyExtractor(BaseEstimator, TransformerMixin):
    """Extract the subject & body from a usenet post in a single pass.

    Takes a sequence of strings and produces a dict of sequences.  Keys are
    `subject` and `body`.
    """
    def fit(self, x, y=None):
        return self

    def transform(self, posts):
        features = np.recarray(shape=(len(posts),),
                               dtype=[('subject', object), ('body', object)])
        for i, text in enumerate(posts):
            headers, _, bod = text.partition('\n\n')
            bod = strip_newsgroup_footer(bod)
            bod = strip_newsgroup_quoting(bod)
            features['body'][i] = bod

            prefix = 'Subject:'
            sub = ''
            for line in headers.split('\n'):
                if line.startswith(prefix):
                    sub = line[len(prefix):]
                    break
            features['subject'][i] = sub

        return features


pipeline = Pipeline([
    # Extract the subject & body
    #('subjectbody', SubjectBodyExtractor()),

    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('get-title', Pipeline([
                ('selector', ItemSelector(key='request_title')),
                ('cv', CountVectorizer(preprocessor = preprocess,
                                       analyzer='char_wb', 
                                       max_df=0.5, ngram_range=(1,3)))
            ])),

            # Pipeline for standard bag-of-words model for body
            ('get-request', Pipeline([
                ('selector', ItemSelector(key='request_text_edit_aware')),
                ('cv', CountVectorizer(preprocessor = preprocess, 
                                       analyzer='char_wb', 
                                       max_df=0.5, ngram_range=(1,3)))
            ])),
            
           #('get-number', Pipeline([
           #     ('selector', ItemSelector(key='number'))
           # ]))

        ]
    )),

    # Use a SVC classifier on the combined features
    ('nb',LogisticRegression(penalty='l2',C=100)),
])





In [18]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      False       0.77      0.78      0.77       762
       True       0.29      0.28      0.29       248

avg / total       0.65      0.66      0.65      1010



In [61]:
#nope, the first one with no preprocessing works best
#the f1 score is good for not getting pizza but bad for getting pizza
#this tells us that more than the text is what matters when a pizza
#wish is granted.

In [19]:
test = pd.read_json('test.json')
for x,y in enumerate(test.columns):
    print(x, y)

0 giver_username_if_known
1 request_id
2 request_text_edit_aware
3 request_title
4 requester_account_age_in_days_at_request
5 requester_days_since_first_post_on_raop_at_request
6 requester_number_of_comments_at_request
7 requester_number_of_comments_in_raop_at_request
8 requester_number_of_posts_at_request
9 requester_number_of_posts_on_raop_at_request
10 requester_number_of_subreddits_at_request
11 requester_subreddits_at_request
12 requester_upvotes_minus_downvotes_at_request
13 requester_upvotes_plus_downvotes_at_request
14 requester_username
15 unix_timestamp_of_request
16 unix_timestamp_of_request_utc


In [20]:
x_fin_test = test[['request_text_edit_aware', 'request_title']]

In [21]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      False       0.77      0.78      0.77       762
       True       0.29      0.28      0.29       248

avg / total       0.65      0.66      0.65      1010



In [83]:
pred1 = pipeline.predict(x_fin_test)

In [70]:
X_test[:5]

request_text_edit_aware  \
1177  Just finished up the last of my classes and ru...   
2637  This is my first time requesting for a pizza. ...   
1915  Hi guys! \n\nI'm a student,  typically strappe...   
1091  It's been a very tight two weeks, and the wife...   
1926  I just got home from work, and im broke, not e...   

                                          request_title  
1177  [REQUEST] Toronto Ontario. Broke student could...  
2637  [REQUEST] GF of 5 years just broke up with me....  
1915  [REQUEST] Bit tight on cash, but more for a la...  
1091  [Request]Columbia, SC. Two chicken breasts and...  
1926  [Request] Home from work and tired in Henderso...

In [71]:
x_fin_test[:5]

request_text_edit_aware  \
0  Hey all! It's about 95 degrees here and our ki...   
1  I didn't know a place like this exists! \n\nI ...   
2  Hi Reddit. Im a single dad having a really rou...   
3  Hi I just moved to Waltham MA from my home sta...   
4  We're just sitting here near indianapolis on o...   

                                       request_title  
0  [request] pregger gf 95 degree house and no fo...  
1  [Request] Lost my job day after labour day, st...  
2                (Request) pizza for my kids please?  
3  [Request] Just moved to a new state(Waltham MA...  
4  [Request] Two girls in between paychecks, we'v...

In [84]:
pred1[:5]

array([False, False,  True, False, False], dtype=bool)

In [89]:
def inter(x):
    return int(x)
vint = np.vectorize(inter)
pred2 = vint(pred1)

In [90]:
pred2[:5]

array([0, 0, 1, 0, 0])

In [101]:
fin_df = pd.concat([test['request_id'],pd.DataFrame(pred2, columns=['requester_received_pizza'])],axis =1)

In [103]:
fin_df.shape

(1631, 2)

In [105]:
fin_df.to_csv('pizza_submission.csv', sep = ',', header=True, index=False)

In [23]:
#Nikki's first three hypotheses: date/time will matter

import datetime
def day_time(x):
    y = ''
    if datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 10:
        y = 'morning'
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 10 & datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 16:
        y = 'midday'
    elif datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour >= 16 & datetime.datetime.fromtimestamp(x['unix_timestamp_of_request_utc']).hour < 21:
        y = 'evening'
    else: 
        y = 'night'
    return y
        
def human_time(a):
    import datetime
    from datetime import date
    import calendar
    ### for the data in raop, return human time.  maybe the time of day matters
    a['human_readable_local_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime('%Y-%m-%d %H:%M:%S')
    a['human_readable_UTC_time'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request_utc']).strftime('%Y-%m-%d %H:%M:%S')
    a['weekday'] = calendar.day_name[datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).weekday()]
    a['month'] = datetime.datetime.fromtimestamp(a['unix_timestamp_of_request']).strftime("%B")
    a['time_of_date'] = day_time(a)
    return a
